In [8]:
# Libraries

import numpy as np
import pandas as pd
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Flatten, Activation, Dropout, Lambda, Cropping2D
from keras.layers.pooling import MaxPool2D
import sklearn
import cv2
import os
import csv
import matplotlib as plt
from sklearn.model_selection import train_test_split

In [ ]:
# Image Flipper

In [ ]:
#Image Reader - Random Examples from each Sample


imageDim = '''TODO-(depth,height,width)''' 

In [ ]:
# Show how much of the Image has to be cropped
top = '''TODO-integer'''
bottom = '''TODO-integer'''
left = '''TODO-integer'''
right = '''TODO-integer'''

croppedImageDim = (imageDim[0],imageDim[1]-(top+bottom),imageDim[2]-(left+right))





In [ ]:
# NVIDIA Architecture + My improvements

def myNet(train_Gen, validation_Gen, numOfTrainSample, numOfValSample, ep):

    #Create the Keras Model
    model = Sequential()

    #Crop Images 
    model.add(Cropping2D(cropping=((top,bottom),(left,right)),input_shape=imageDim))

    #Normalization and Mean center
    model.add(Lambda(lambda x: (x/255.0)-0.5)) # TODO maybe also output_shape=croppedImageDim OR No size Paramenters at all?

    #Convolutional Layer 1
    model.add(Convolution2D(24,5,5))
    model.add(Activation('relu'))

    #Convolutional Layer 2
    model.add(Convolution2D(36,5,5))
    model.add(Activation('relu'))

    #Convolutional Layer 3
    model.add(Convolution2D(48,5,5))
    model.add(Activation('relu'))

    #Convolutional Layer 4
    model.add(Convolution2D(64,3,3))
    model.add(Activation('relu'))

    #Convolutional Layer 5
    model.add(Convolution2D(64,3,3))
    model.add(Activation('relu'))

    #Flatten
    model.add(Flatten())

    #Fully Connected Layer 1
    model.add(Dense(100))
    model.add(Dropout(0.5))
    model.add(Activation('relu'))

    #Fully Connected Layer 2
    model.add(Dense(50))
    model.add(Dropout(0.5)) #TODO ?

    #Fully Connected Layer 3
    model.add(Dense(10))
    model.add(Activation('relu'))

    #Final Output
    model.add(Dense(1))

    model.compile(loss='mse',optimizer='adam')
    historyObj = model.fit_generator(X_Train_Gen, samples_per_epoch=numOfTrainSample,
                        validation_data=validation_Gen, nb_val_samples=numOfValSample
                        nb_epoch = ep, verbose=1)

    return historyObj # Can be used for Visualing historyObj.history['loss'] and historyObj.history['loss'] && also check out Keys: print(history_object.history.keys())
